# 第 4 讲：求积问题

## Newton-Cotes

In [55]:
import numpy as np
import sympy as sy

In [95]:
def factorial(num):
    res = 1
    for x in range(1, num+1):
        res *= x
    return res

In [96]:
def mul(n, i, t):
    res = 1
    for j in range(0, n+1):
        if j != i:
            res *= (t - j)
    return res

In [97]:
def dt(a, b, i, f):
    t = np.linspace(a, b, 1000)
    return sum(((b-a) / 1000) * f(b-a, i, t))

In [98]:
def C(n, i):
    return ( (-1) ** (n - i) ) / (factorial(i) * factorial(n - i)) * dt(0, n, i, mul)

In [99]:
def f(x):
    return 1/ (1 + x)

In [103]:
def NC(f, xs, n):
    x = np.linspace(xvalues[0], xs[-1], n+1)
    h = x[1] - x[0]
    total = 0
    for i in range(0, n+1):
        total += C(n, i) * f(x[i])
    return h * total

In [104]:
xvalues = [0, 1]

In [105]:
NC(f, xvalues, 4)

0.6932314905381517

## 梯形公式

令Newton-Cotes的n变为1，我们就得到了梯形公式

递归写法

In [230]:
def trapezoidal(f, a, b, n):
    if n == 1:
        return (b-a) * (f(a) + f(b)) / 2
    else:
        if n % 2 == 0:
            h = (b-a) / n
            x = [a + i*h for i in range(n+1)]
        else:
            h = (b-a) / (n+1)
            x = [a + i*h for i in range(n+1)]
        s1 = trapezoidal(f, a, x[n//2], n//2)
        s2 = trapezoidal(f, x[n//2], b, n//2)
        return s1 + s2

In [231]:
trapezoidal_rule(f, 0, 1, 4)

0.6970238095238095

## Romberg

In [359]:
def Romberg(f, a, b, n):
    R = [[0 for i in range(n + 1)] for j in range(n+1)]
    R[1][1] = 1/2 * (b - a) * ( f(a) + f(b) )
    # print(R[1][1])
    for i in range(2, n + 1):
        h = (b - a) / (2 ** (i - 1))
        R[i][1] = 1/2 * R[i-1][1] +  h * sum(( f(a + (2 * j - 1) * h ) for j in range(1, 2 ** (i - 2) + 1) ))
        print(f"{i} {1}")
        print(R[i][1])
        for k in range(2, i + 1):
            R[i][k] = (4 ** (k - 1) * R[i][k-1] - R[i-1][k-1]) / (4 ** (k - 1) - 1)
            print(f"{i} {k}")            
            print(R[i][k])

In [360]:
Romberg(f, 0, 3.14, 6)

2 1
1.5712497347424226
2 2
2.094166157963602
3 1
1.8962240978279428
3 2
2.0045488855231164
3 3
1.9985744006937507
4 1
1.974256537721767
4 2
2.0002673510197084
4 3
1.9999819153861478
4 4
2.0000042568892016
5 1
1.993575601232516
5 2
2.0000152890694323
5 3
1.999998484939414
5 4
1.999998747948196
5 5
1.9999987263445058
6 1
1.998393722559025
6 2
1.9999997630011945
6 3
1.9999987279299787
6 4
1.9999987317869719
6 5
1.9999987317235945
6 6
1.9999987317288528


## Gauss-Legendre

In [60]:
def Legendre(N, x):
    p0 = 1
    p1 = x
    sympy.simplify(x)
    for i in range(1,N):
        p = (2*i+1)/(i+1)*x*p1 - i/(i+1)*p0
        p0 = p1
        p1 = p
    return sympy.simplify(p1)

In [76]:
Legendre(2, sy.symbols('x'))

1.5*x**2 - 0.5

In [77]:
print(sy.solve(Legendre(2, sy.symbols('x'))))
print(sy.solve(Legendre(3, sy.symbols('x'))))
print(sy.solve(Legendre(4, sy.symbols('x'))))
print(sy.solve(Legendre(5, sy.symbols('x'))))
print(sy.solve(Legendre(6, sy.symbols('x'))))
print(sy.solve(Legendre(7, sy.symbols('x'))))
print(sy.solve(Legendre(8, sy.symbols('x'))))

[-0.577350269189626, 0.577350269189626]
[-0.774596669241483, 0.0, 0.774596669241483]
[-0.861136311594053, -0.339981043584856, 0.339981043584856, 0.861136311594053]
[-0.906179845938664, -0.538469310105683, 0.0, 0.538469310105683, 0.906179845938664]
[-0.932469514203152 + 4.16328475581762e-103*I, -0.661209386466264 + 5.90432806037835e-103*I, -0.238619186083197 - 1.70461700631958e-102*I, 0.238619186083197 - 1.36369360505566e-101*I, 0.661209386466264 + 9.44692489660537e-102*I, 0.932469514203152 - 4.16328475581762e-103*I]
[0.0, -0.949107912342758 - 8.475143568547e-103*I, -0.741531185599394 + 2.64862748557095e-102*I, -0.405845151377397 - 3.62403039716727e-103*I, 0.405845151377397 + 3.62403039716727e-103*I, 0.741531185599394 + 9.93235307089107e-103*I, 0.949107912342758 - 2.11878589213675e-103*I]
[-0.960289856497536, -0.796666477413627, -0.525532409916329, -0.183434642495650, 0.183434642495650, 0.525532409916329, 0.796666477413627, 0.960289856497536]


再来求1/(1+x)在0到1上的积分